In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import copy
import warnings
import sklearn.preprocessing.
# from scipy import stats
# from scipy.stats import t
from IPython.display import display
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import accuracy_score
# from sklearn.metrics import roc_curve
# from sklearn.metrics import auc
# from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.ensemble import BaggingClassifier
# from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn import naive_bayes
# from sklearn import neighbors
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
# Loading the data set and splitting off the target feature, 'Class'
image_df = pd.read_table("uspsdata.txt", header=None)
class_df = pd.read_table("uspscl.txt", header=None)
display(image_df.head(5))
display(class_df.head(5))

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.0,0.0,0.00,0.00,0.95,11.91,49.02,106.96,152.14,157.77,...,145.84,154.03,152.60,142.38,125.40,100.57,69.80,38.67,14.58,2.80
1,0.0,0.0,3.31,19.03,51.45,87.66,106.14,102.35,96.37,106.66,...,125.28,141.59,145.34,135.52,110.81,74.17,36.65,11.53,1.81,0.07
2,0.0,0.0,0.02,1.86,12.63,42.03,84.17,104.87,79.06,32.90,...,158.29,150.08,134.27,113.54,85.57,52.14,23.76,7.75,1.76,0.23
3,0.0,0.0,0.00,0.01,5.93,33.21,75.00,85.97,51.36,14.25,...,157.26,156.53,139.58,104.68,63.12,28.38,8.15,1.11,0.02,0.00
4,0.0,0.0,0.94,8.57,32.06,63.96,74.10,50.12,18.39,2.83,...,93.12,87.66,59.11,26.47,8.72,2.48,0.42,0.00,0.00,0.00


,0
0,-1
1,-1
2,1
3,1
4,1


In [20]:
scaler = StandardScaler()
scaled_image_df = scaler.fit_transform(image_df)

In [21]:
img_train, img_test, class_train, class_test = train_test_split(scaled_image_df,
                                                                class_df,
                                                                test_size=0.20,
                                                                random_state=123)

In [27]:
svm_clf = LinearSVC(C=1, loss="hinge")

In [37]:
svm_clf.fit(img_train, np.ravel(class_train))


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [36]:
# np.array(class_train).shape
# svm_clf = Pipeline((
#                     ("scaler", StandardScaler()),
#                     ("linear_svc", LinearSVC(C=1, loss="hinge")),
#                   ))

help(np.ravel)

Help on function ravel in module numpy.core.fromnumeric:

ravel(a, order='C')
    Return a contiguous flattened array.
    
    A 1-D array, containing the elements of the input, is returned.  A copy is
    made only if needed.
    
    As of NumPy 1.10, the returned array will have the same type as the input
    array. (for example, a masked array will be returned for a masked array
    input)
    
    Parameters
    ----------
    a : array_like
        Input array.  The elements in `a` are read in the order specified by
        `order`, and packed as a 1-D array.
    order : {'C','F', 'A', 'K'}, optional
    
        The elements of `a` are read using this index order. 'C' means
        to index the elements in row-major, C-style order,
        with the last axis index changing fastest, back to the first
        axis index changing slowest.  'F' means to index the elements
        in column-major, Fortran-style order, with the
        first index changing fastest, and the last index

sklearn.pipeline.Pipeline

In [24]:
# svm_clf.fit(img_train_scaled, class_train)

In [9]:
help(LinearSVC)

Help on class LinearSVC in module sklearn.svm.classes:

class LinearSVC(sklearn.base.BaseEstimator, sklearn.linear_model.base.LinearClassifierMixin, sklearn.linear_model.base.SparseCoefMixin)
 |  Linear Support Vector Classification.
 |  
 |  Similar to SVC with parameter kernel='linear', but implemented in terms of
 |  liblinear rather than libsvm, so it has more flexibility in the choice of
 |  penalties and loss functions and should scale better to large numbers of
 |  samples.
 |  
 |  This class supports both dense and sparse input and the multiclass support
 |  is handled according to a one-vs-the-rest scheme.
 |  
 |  Read more in the :ref:`User Guide <svm_classification>`.
 |  
 |  Parameters
 |  ----------
 |  penalty : string, 'l1' or 'l2' (default='l2')
 |      Specifies the norm used in the penalization. The 'l2'
 |      penalty is the standard used in SVC. The 'l1' leads to ``coef_``
 |      vectors that are sparse.
 |  
 |  loss : string, 'hinge' or 'squared_hinge' (defau